In [ ]:
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib
import cv2
import csv

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Sequence_to_sequence/h5_models/')
model_name = "UNet_Xception_style_model_24.h5" 
# model_name = 'saeed_model.h5'

model = load_model(model_name, compile=False)
model.summary()

In [ ]:
exp_gt = 'label_L3_S4_B_333x333p_50kHz_10HC_18Vpp_x10_pzt.png'

In [ ]:
input_dir = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Experimental/median_filter_output_exp/' 
target_dir = "/aijjeh_odroid_sensors/aidd/data/interim/exp/new_exp/"

In [ ]:
height =  512 
width =  512 
img_size = (height, width)
num_classes = 2
batch_size = 1
time_stamps = 24 
input_img_paths_total = [None]*1

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
path_to_csv = '/home/aijjeh/aijjeh_rexio_share/PhD/cmap_flipped_jet256.csv'
cmap = matplotlib.colors.ListedColormap(["blue", "green", "red"], name=path_to_csv, N=None)

In [ ]:
my_list = os.listdir(input_dir)
my_list.sort(key=natural_keys)
print(len(my_list))

In [ ]:
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname) 
        for fname in os.listdir(input_dir)    
        if fname.endswith(".png")
    ])
input_img_paths.sort(key=natural_keys)
input_img_paths_total[0]= input_img_paths
print((input_img_paths_total))

In [ ]:
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(exp_gt) and not fname.startswith(".")
    ]
)
target_img_paths.sort(key=natural_keys)
print((target_img_paths))

In [ ]:
class Full_wavefield_frames(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths_total_, target_img_paths_, time_stamps, frame_init):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths_total = input_img_paths_total_
        self.target_img_paths = target_img_paths_
        self.time_stamps = time_stamps
        self.frame_num = frame_init
    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size        
        batch_input_img_paths = self.input_img_paths_total[i:i+self.batch_size]      
        batch_target_img_paths = self.target_img_paths[i:i+self.batch_size] 
        
        x = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (1,), dtype="float32") #  
        
        for batch_num in range(self.batch_size):
            batch_input_img_paths = batch_input_img_paths[batch_num][self.frame_num:self.frame_num+self.time_stamps] 
            for j, path in enumerate(batch_input_img_paths):
                img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
                img = np.expand_dims(img,2)
                img = img / 255.0
                x[batch_num][j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
            img = np.expand_dims(img, 2)
            img = img / 255.0
            y[j] = img                
        return x, y

In [ ]:
test_input_img_paths = input_img_paths_total[:]
test_target_img_paths = target_img_paths[:]

In [ ]:
def calc_IoU(predicted_image, truth_img, frame):
    predicted_image = np.asarray(predicted_image) # 
    predicted_image = predicted_image.astype('float64')
    ret, predicted_image = cv2.threshold(predicted_image, .5, 1.0, cv2.THRESH_BINARY)
    truth_img = np.asarray(truth_img)# 
    truth_img = truth_img.astype('float64')
    ret_, truth_img = cv2.threshold(truth_img, .5, 1.0, cv2.THRESH_BINARY)

    truth_img = truth_img[:,:,0]
    InterSectionArray = cv2.bitwise_and(predicted_image, truth_img)
    UnionArray = cv2.bitwise_or(predicted_image, truth_img)
    I1 = np.count_nonzero(InterSectionArray)
    U = np.count_nonzero(UnionArray)
    IoU = I1 / U
    print('frame_%d_iou : ' % frame,IoU)
    return IoU

In [ ]:
total_IoU  = 0

In [ ]:
exp_path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Experimental/Exp_results_Full_wavefield/'
os.chdir(exp_path)

In [ ]:
def plot_res(img, ext, frame):
    ############################################################################################################
    plt.figure(figsize=(1, 1), dpi=512)
    plt.gca().set_axis_off()
    plt.axis('off')
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    ############################################################################################################
    # plt.imshow(img, cmap=cmap)
    img = np.asarray(img)
    # img = img *255
    # img = img.astype(np.uint8)
    cv2.imwrite('frame_%d_exp.png' % frame, img)
    # plt.savefig('frame_%d_case_%d.png' % (frame, case))
    # np.save('frame_%d_case_%d' % (frame, case), img)
    # plt.show()
    plt.close('all')

In [ ]:
list_frames = []
list_iou = []

In [ ]:
for frame_init in range(0, len(my_list)-24):
    val_gen = Full_wavefield_frames(batch_size, img_size, input_img_paths_total, target_img_paths, time_stamps, frame_init)
    val_preds = model.predict(val_gen)
    
    # Display ground-truth target mask
    
    img_GT_ = PIL.ImageOps.autocontrast(load_img(test_target_img_paths[0], target_size= img_size))
    
    # Display mask predicted by our model
    
    mask = val_preds[0]
    img_pred_ = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask)) # 
    plot_res(keras.preprocessing.image.array_to_img(mask), 'predicted_output', frame_init)
    iou = calc_IoU(img_pred_, img_GT_, frame_init)       
    total_IoU = total_IoU + iou
    list_frames.append(frame_init)
    list_iou.append(iou)